# Week 3 Task 5.1

In [1]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv("rawpvr_2018-02-01_28d_1083 TueFri.csv")
data["Date"] = pd.to_datetime(data["Date"], format="%Y-%m-%d %H:%M:%S.%f")

data["Hour"] = data["Date"].dt.hour

tue_data = data[(data["Flags"]==2) & (data["Hour"]>=7) & (data["Hour"]<19)]

## Gap(s) Completeness

In [7]:
gaps_data_array = tue_data["Gap (s)"].values
print("Gap (s) values array", gaps_data_array)
gaps_data_len = len(gaps_data_array)
gaps_data_nan_len = np.isnan(gaps_data_array).sum()
print("Number of cells ", gaps_data_len)
print("Number of empty cells", gaps_data_nan_len)

Gap (s) values array [  nan   nan 1.992 ... 5.182 7.307 1.133]
Number of cells  201125
Number of empty cells 3949


In [8]:
gaps_column_completeness = ((gaps_data_len - gaps_data_nan_len)*100) / gaps_data_len
print("The completeness of the \'Gap (s)\' column ", gaps_column_completeness)

The completeness of the 'Gap (s)' column  98.03654443753885


## Headway (s) completeness

In [9]:
headways_data_no_nan_len = tue_data.count()["Headway (s)"]
headways_data_len = len(tue_data)
print("Number of non-empty cells", headways_data_no_nan_len)
print("Number of cells", headways_data_len)

Number of non-empty cells 199044
Number of cells 201125


In [10]:
headways_column_completeness = (headways_data_no_nan_len*100) / headways_data_len
print("The level of completeness of the \'Headway (s) \' column: ", headways_column_completeness)

The level of completeness of the 'Headway (s) ' column:  98.96532007458049


# Week 3 Task 5.2

In [11]:
import pandas as pd
import numpy as np

In [13]:
data = pd.read_csv("rawpvr_2018-02-01_28d_1083 TueFri.csv")
data["Date"] = pd.to_datetime(data["Date"], format="%Y-%m-%d %H:%M:%S.%f")
data["Hour"] = data["Date"].dt.hour
tue_data = data[(data["Direction Name"]=="North") & (data["Lane Name"]=="NB_MID") & (data["Hour"]>=7) & (data["Hour"]<19) & (data["Flags"]==2)]

## Gap (s) median from 7:00 to 19:00 and fill in

In [18]:
tue_data[tue_data["Hour"]==7].dropna(axis=0, how="any", subset=["Gap (s)"]).sort_values("Gap (s)")

,Date,Lane,Lane Name,Direction,Direction Name,Speed (mph),Headway (s),Gap (s),Flags,Flag Text,Hour
198096,2018-02-13 07:36:59.030,2,NB_MID,1,North,11.807,2.463,0.025,2,Tuesday,7
450317,2018-02-27 07:16:28.070,2,NB_MID,1,North,26.098,1.714,0.094,2,Tuesday,7
197526,2018-02-13 07:30:30.040,2,NB_MID,1,North,27.962,3.800,0.143,2,Tuesday,7
325982,2018-02-20 07:44:04.070,2,NB_MID,1,North,3.729,2.100,0.220,2,Tuesday,7
70825,2018-02-06 07:28:08.050,2,NB_MID,1,North,19.884,0.500,0.279,2,Tuesday,7
...,...,...,...,...,...,...,...,...,...,...,...
323543,2018-02-20 07:17:04.020,2,NB_MID,1,North,35.417,36.100,35.605,2,Tuesday,7
195734,2018-02-13 07:08:06.080,2,NB_MID,1,North,33.554,38.900,38.571,2,Tuesday,7
452091,2018-02-27 07:39:03.000,2,NB_MID,1,North,9.321,3.840,39.433,2,Tuesday,7
326480,2018-02-20 07:49:53.020,2,NB_MID,1,North,9.321,42.900,42.197,2,Tuesday,7


In [19]:
gaps_median_array = []
for i in range(7,19):
    gaps_data_sorted_array = tue_data[tue_data["Hour"]==i].dropna(axis=0, how="any", subset=["Gap (s)"]).sort_values("Gap (s)")["Gap (s)"].values
    if len(gaps_data_sorted_array) % 2 == 1:
        temp_median = gaps_data_sorted_array[int(len(gaps_data_sorted_array)/2)]
    elif len(gaps_data_sorted_array) % 2 == 0:
        temp_median = (gaps_data_sorted_array[int(len(gaps_data_sorted_array)/2-1)] + gaps_data_sorted_array[int(len(gaps_data_sorted_array)/2)])/2
        
    print("Median value of Gap (s) between %d:00 and %d:00: " % (i, i+1), temp_median)
    gaps_median_array.append(temp_median)

Median value of Gap (s) between 7:00 and 8:00:  1.834
Median value of Gap (s) between 8:00 and 9:00:  2.1020000000000003
Median value of Gap (s) between 9:00 and 10:00:  2.08
Median value of Gap (s) between 10:00 and 11:00:  2.5835
Median value of Gap (s) between 11:00 and 12:00:  2.7785
Median value of Gap (s) between 12:00 and 13:00:  2.795
Median value of Gap (s) between 13:00 and 14:00:  2.7560000000000002
Median value of Gap (s) between 14:00 and 15:00:  2.8049999999999997
Median value of Gap (s) between 15:00 and 16:00:  2.577
Median value of Gap (s) between 16:00 and 17:00:  2.3225
Median value of Gap (s) between 17:00 and 18:00:  2.187
Median value of Gap (s) between 18:00 and 19:00:  2.228


In [22]:
for i in range(7,19):
    temp_fill_in_index = list(tue_data[(np.isnan(tue_data["Gap (s)"])) & (tue_data["Hour"]==i)].index)
    data.loc[temp_fill_in_index,"Gap (s)"] = gaps_median_array[i-7]

In [23]:
tue_data.loc[tue_data[(np.isnan(tue_data["Gap (s)"]) & (tue_data["Hour"]==7))].index].head(5)

,Date,Lane,Lane Name,Direction,Direction Name,Speed (mph),Headway (s),Gap (s),Flags,Flag Text,Hour
68517,2018-02-06 07:00:01.160,2,NB_MID,1,North,27.962,1.720,NaN,2,Tuesday,7
68547,2018-02-06 07:00:15.150,2,NB_MID,1,North,27.340,1.841,NaN,2,Tuesday,7
68888,2018-02-06 07:05:29.000,2,NB_MID,1,North,34.798,NaN,NaN,2,Tuesday,7
69204,2018-02-06 07:10:20.020,2,NB_MID,1,North,35.417,NaN,NaN,2,Tuesday,7
69653,2018-02-06 07:15:16.160,2,NB_MID,1,North,51.575,3.947,NaN,2,Tuesday,7


In [24]:
data.loc[tue_data[(np.isnan(tue_data["Gap (s)"]) & (tue_data["Hour"]==7))].index].head(5)

,Date,Lane,Lane Name,Direction,Direction Name,Speed (mph),Headway (s),Gap (s),Flags,Flag Text,Hour
68517,2018-02-06 07:00:01.160,2,NB_MID,1,North,27.962,1.720,1.834,2,Tuesday,7
68547,2018-02-06 07:00:15.150,2,NB_MID,1,North,27.340,1.841,1.834,2,Tuesday,7
68888,2018-02-06 07:05:29.000,2,NB_MID,1,North,34.798,NaN,1.834,2,Tuesday,7
69204,2018-02-06 07:10:20.020,2,NB_MID,1,North,35.417,NaN,1.834,2,Tuesday,7
69653,2018-02-06 07:15:16.160,2,NB_MID,1,North,51.575,3.947,1.834,2,Tuesday,7


In [25]:
tue_data.loc[tue_data[(np.isnan(tue_data["Gap (s)"]) & (tue_data["Hour"]==8))].index].head(5)

,Date,Lane,Lane Name,Direction,Direction Name,Speed (mph),Headway (s),Gap (s),Flags,Flag Text,Hour
73854,2018-02-06 08:00:02.000,2,NB_MID,1,North,16.777,3.267,NaN,2,Tuesday,8
73875,2018-02-06 08:00:16.000,2,NB_MID,1,North,16.155,3.392,NaN,2,Tuesday,8
74353,2018-02-06 08:05:40.000,2,NB_MID,1,North,17.399,NaN,NaN,2,Tuesday,8
74789,2018-02-06 08:10:15.150,2,NB_MID,1,North,15.533,2.664,NaN,2,Tuesday,8
75106,2018-02-06 08:13:55.040,2,NB_MID,1,North,6.214,0.900,NaN,2,Tuesday,8


In [26]:
data.loc[tue_data[(np.isnan(tue_data["Gap (s)"]) & (tue_data["Hour"]==8))].index].head(5)

,Date,Lane,Lane Name,Direction,Direction Name,Speed (mph),Headway (s),Gap (s),Flags,Flag Text,Hour
73854,2018-02-06 08:00:02.000,2,NB_MID,1,North,16.777,3.267,2.102,2,Tuesday,8
73875,2018-02-06 08:00:16.000,2,NB_MID,1,North,16.155,3.392,2.102,2,Tuesday,8
74353,2018-02-06 08:05:40.000,2,NB_MID,1,North,17.399,NaN,2.102,2,Tuesday,8
74789,2018-02-06 08:10:15.150,2,NB_MID,1,North,15.533,2.664,2.102,2,Tuesday,8
75106,2018-02-06 08:13:55.040,2,NB_MID,1,North,6.214,0.900,2.102,2,Tuesday,8


In [39]:
tue_data.loc[tue_data[(np.isnan(tue_data["Gap (s)"]) & (tue_data["Hour"]==12))].index].head(5)

,Date,Lane,Lane Name,Direction,Direction Name,Speed (mph),Headway (s),Gap (s),Flags,Flag Text,Hour
89354,2018-02-06 12:00:00.110,2,NB_MID,1,North,32.932,4.075,NaN,2,Tuesday,12
89372,2018-02-06 12:00:19.090,2,NB_MID,1,North,38.525,NaN,NaN,2,Tuesday,12
89623,2018-02-06 12:05:23.050,2,NB_MID,1,North,41.010,NaN,NaN,2,Tuesday,12
89908,2018-02-06 12:10:15.110,2,NB_MID,1,North,35.417,NaN,NaN,2,Tuesday,12
90216,2018-02-06 12:15:16.020,2,NB_MID,1,North,36.661,2.502,NaN,2,Tuesday,12


In [40]:
data.loc[tue_data[(np.isnan(tue_data["Gap (s)"]) & (tue_data["Hour"]==12))].index].head(5)

,Date,Lane,Lane Name,Direction,Direction Name,Speed (mph),Headway (s),Gap (s),Flags,Flag Text,Hour
89354,2018-02-06 12:00:00.110,2,NB_MID,1,North,32.932,4.075,2.795,2,Tuesday,12
89372,2018-02-06 12:00:19.090,2,NB_MID,1,North,38.525,NaN,2.795,2,Tuesday,12
89623,2018-02-06 12:05:23.050,2,NB_MID,1,North,41.010,NaN,2.795,2,Tuesday,12
89908,2018-02-06 12:10:15.110,2,NB_MID,1,North,35.417,NaN,2.795,2,Tuesday,12
90216,2018-02-06 12:15:16.020,2,NB_MID,1,North,36.661,2.502,2.795,2,Tuesday,12


In [30]:
tue_data.loc[tue_data[(np.isnan(tue_data["Gap (s)"]) & (tue_data["Hour"]==13))].index].head(5)

,Date,Lane,Lane Name,Direction,Direction Name,Speed (mph),Headway (s),Gap (s),Flags,Flag Text,Hour
92784,2018-02-06 13:00:01.170,2,NB_MID,1,North,32.310,1.627,NaN,2,Tuesday,13
92799,2018-02-06 13:00:16.160,2,NB_MID,1,North,29.205,2.336,NaN,2,Tuesday,13
93104,2018-02-06 13:05:16.030,2,NB_MID,1,North,32.932,2.785,NaN,2,Tuesday,13
93413,2018-02-06 13:10:16.000,2,NB_MID,1,North,27.962,1.800,NaN,2,Tuesday,13
93720,2018-02-06 13:15:19.010,2,NB_MID,1,North,30.447,5.437,NaN,2,Tuesday,13


In [32]:
data.loc[tue_data[(np.isnan(tue_data["Gap (s)"]) & (tue_data["Hour"]==13))].index].head(5)

,Date,Lane,Lane Name,Direction,Direction Name,Speed (mph),Headway (s),Gap (s),Flags,Flag Text,Hour
92784,2018-02-06 13:00:01.170,2,NB_MID,1,North,32.310,1.627,2.756,2,Tuesday,13
92799,2018-02-06 13:00:16.160,2,NB_MID,1,North,29.205,2.336,2.756,2,Tuesday,13
93104,2018-02-06 13:05:16.030,2,NB_MID,1,North,32.932,2.785,2.756,2,Tuesday,13
93413,2018-02-06 13:10:16.000,2,NB_MID,1,North,27.962,1.800,2.756,2,Tuesday,13
93720,2018-02-06 13:15:19.010,2,NB_MID,1,North,30.447,5.437,2.756,2,Tuesday,13


In [35]:
tue_data.loc[tue_data[(np.isnan(tue_data["Gap (s)"]) & (tue_data["Hour"]==17))].index].head(5)

,Date,Lane,Lane Name,Direction,Direction Name,Speed (mph),Headway (s),Gap (s),Flags,Flag Text,Hour
110218,2018-02-06 17:00:00.010,2,NB_MID,1,North,26.098,1.500,NaN,2,Tuesday,17
110251,2018-02-06 17:00:17.020,2,NB_MID,1,North,22.991,4.962,NaN,2,Tuesday,17
110448,2018-02-06 17:02:21.000,2,NB_MID,1,North,7.456,2.400,NaN,2,Tuesday,17
110595,2018-02-06 17:03:57.080,2,NB_MID,1,North,21.126,1.853,NaN,2,Tuesday,17
110712,2018-02-06 17:05:17.020,2,NB_MID,1,North,28.584,1.996,NaN,2,Tuesday,17


In [36]:
data.loc[tue_data[(np.isnan(tue_data["Gap (s)"]) & (tue_data["Hour"]==17))].index].head(5)

,Date,Lane,Lane Name,Direction,Direction Name,Speed (mph),Headway (s),Gap (s),Flags,Flag Text,Hour
110218,2018-02-06 17:00:00.010,2,NB_MID,1,North,26.098,1.500,2.187,2,Tuesday,17
110251,2018-02-06 17:00:17.020,2,NB_MID,1,North,22.991,4.962,2.187,2,Tuesday,17
110448,2018-02-06 17:02:21.000,2,NB_MID,1,North,7.456,2.400,2.187,2,Tuesday,17
110595,2018-02-06 17:03:57.080,2,NB_MID,1,North,21.126,1.853,2.187,2,Tuesday,17
110712,2018-02-06 17:05:17.020,2,NB_MID,1,North,28.584,1.996,2.187,2,Tuesday,17


In [37]:
tue_data.loc[tue_data[(np.isnan(tue_data["Gap (s)"]) & (tue_data["Hour"]==18))].index].head(5)

,Date,Lane,Lane Name,Direction,Direction Name,Speed (mph),Headway (s),Gap (s),Flags,Flag Text,Hour
115619,2018-02-06 18:00:01.190,2,NB_MID,1,North,41.010,NaN,NaN,2,Tuesday,18
115652,2018-02-06 18:00:16.020,2,NB_MID,1,North,29.205,1.800,NaN,2,Tuesday,18
115895,2018-02-06 18:03:03.180,2,NB_MID,1,North,32.932,1.902,NaN,2,Tuesday,18
116459,2018-02-06 18:10:17.020,2,NB_MID,1,North,36.661,NaN,NaN,2,Tuesday,18
116886,2018-02-06 18:15:15.140,2,NB_MID,1,North,26.718,2.302,NaN,2,Tuesday,18


In [38]:
data.loc[tue_data[(np.isnan(tue_data["Gap (s)"]) & (tue_data["Hour"]==18))].index].head(5)

,Date,Lane,Lane Name,Direction,Direction Name,Speed (mph),Headway (s),Gap (s),Flags,Flag Text,Hour
115619,2018-02-06 18:00:01.190,2,NB_MID,1,North,41.010,NaN,2.228,2,Tuesday,18
115652,2018-02-06 18:00:16.020,2,NB_MID,1,North,29.205,1.800,2.228,2,Tuesday,18
115895,2018-02-06 18:03:03.180,2,NB_MID,1,North,32.932,1.902,2.228,2,Tuesday,18
116459,2018-02-06 18:10:17.020,2,NB_MID,1,North,36.661,NaN,2.228,2,Tuesday,18
116886,2018-02-06 18:15:15.140,2,NB_MID,1,North,26.718,2.302,2.228,2,Tuesday,18


## Headway (s) median from 7:00 to 19:00 and fill in

In [42]:
headways_median = tue_data.groupby(["Hour"])["Headway (s)"].median()
headways_median

Hour
7     2.7220
8     3.1910
9     2.7200
10    3.0000
11    3.2100
12    3.1600
13    3.1020
14    3.1330
15    2.9200
16    2.8530
17    2.9065
18    2.7900
Name: Headway (s), dtype: float64

In [43]:
headways_median_array = list(headways_median)

for i in range(7,19):
    temp_fill_in_index = list(tue_data[(np.isnan(tue_data["Headway (s)"])) & (tue_data["Hour"]==i)].index)
    data.loc[temp_fill_in_index,"Headway (s)"] = headways_median_array[i-7]

In [44]:
tue_data.loc[tue_data[(np.isnan(tue_data["Headway (s)"])) & (tue_data["Hour"]==7)].index].head(5)

,Date,Lane,Lane Name,Direction,Direction Name,Speed (mph),Headway (s),Gap (s),Flags,Flag Text,Hour
68888,2018-02-06 07:05:29.000,2,NB_MID,1,North,34.798,NaN,NaN,2,Tuesday,7
69204,2018-02-06 07:10:20.020,2,NB_MID,1,North,35.417,NaN,NaN,2,Tuesday,7
72419,2018-02-06 07:45:27.040,2,NB_MID,1,North,7.456,NaN,NaN,2,Tuesday,7
72881,2018-02-06 07:50:17.000,2,NB_MID,1,North,25.476,NaN,NaN,2,Tuesday,7
195243,2018-02-13 07:00:23.040,2,NB_MID,1,North,39.768,NaN,NaN,2,Tuesday,7


In [45]:
data.loc[tue_data[(np.isnan(tue_data["Headway (s)"])) & (tue_data["Hour"]==7)].index].head(5)

,Date,Lane,Lane Name,Direction,Direction Name,Speed (mph),Headway (s),Gap (s),Flags,Flag Text,Hour
68888,2018-02-06 07:05:29.000,2,NB_MID,1,North,34.798,2.722,1.834,2,Tuesday,7
69204,2018-02-06 07:10:20.020,2,NB_MID,1,North,35.417,2.722,1.834,2,Tuesday,7
72419,2018-02-06 07:45:27.040,2,NB_MID,1,North,7.456,2.722,1.834,2,Tuesday,7
72881,2018-02-06 07:50:17.000,2,NB_MID,1,North,25.476,2.722,1.834,2,Tuesday,7
195243,2018-02-13 07:00:23.040,2,NB_MID,1,North,39.768,2.722,1.834,2,Tuesday,7


In [46]:
tue_data.loc[tue_data[(np.isnan(tue_data["Headway (s)"])) & (tue_data["Hour"]==8)].index].head(5)

,Date,Lane,Lane Name,Direction,Direction Name,Speed (mph),Headway (s),Gap (s),Flags,Flag Text,Hour
74353,2018-02-06 08:05:40.000,2,NB_MID,1,North,17.399,NaN,NaN,2,Tuesday,8
200149,2018-02-13 08:00:02.050,2,NB_MID,1,North,6.214,NaN,NaN,2,Tuesday,8
204738,2018-02-13 08:55:20.020,2,NB_MID,1,North,33.554,NaN,NaN,2,Tuesday,8


In [47]:
data.loc[tue_data[(np.isnan(tue_data["Headway (s)"])) & (tue_data["Hour"]==8)].index].head(5)

,Date,Lane,Lane Name,Direction,Direction Name,Speed (mph),Headway (s),Gap (s),Flags,Flag Text,Hour
74353,2018-02-06 08:05:40.000,2,NB_MID,1,North,17.399,3.191,2.102,2,Tuesday,8
200149,2018-02-13 08:00:02.050,2,NB_MID,1,North,6.214,3.191,2.102,2,Tuesday,8
204738,2018-02-13 08:55:20.020,2,NB_MID,1,North,33.554,3.191,2.102,2,Tuesday,8


In [48]:
tue_data.loc[tue_data[(np.isnan(tue_data["Headway (s)"])) & (tue_data["Hour"]==12)].index].head(5)

,Date,Lane,Lane Name,Direction,Direction Name,Speed (mph),Headway (s),Gap (s),Flags,Flag Text,Hour
89372,2018-02-06 12:00:19.090,2,NB_MID,1,North,38.525,NaN,NaN,2,Tuesday,12
89623,2018-02-06 12:05:23.050,2,NB_MID,1,North,41.010,NaN,NaN,2,Tuesday,12
89908,2018-02-06 12:10:15.110,2,NB_MID,1,North,35.417,NaN,NaN,2,Tuesday,12
90503,2018-02-06 12:20:19.030,2,NB_MID,1,North,34.798,NaN,NaN,2,Tuesday,12
90792,2018-02-06 12:25:34.090,2,NB_MID,1,North,42.253,NaN,NaN,2,Tuesday,12


In [49]:
data.loc[tue_data[(np.isnan(tue_data["Headway (s)"])) & (tue_data["Hour"]==12)].index].head(5)

,Date,Lane,Lane Name,Direction,Direction Name,Speed (mph),Headway (s),Gap (s),Flags,Flag Text,Hour
89372,2018-02-06 12:00:19.090,2,NB_MID,1,North,38.525,3.16,2.795,2,Tuesday,12
89623,2018-02-06 12:05:23.050,2,NB_MID,1,North,41.010,3.16,2.795,2,Tuesday,12
89908,2018-02-06 12:10:15.110,2,NB_MID,1,North,35.417,3.16,2.795,2,Tuesday,12
90503,2018-02-06 12:20:19.030,2,NB_MID,1,North,34.798,3.16,2.795,2,Tuesday,12
90792,2018-02-06 12:25:34.090,2,NB_MID,1,North,42.253,3.16,2.795,2,Tuesday,12


In [50]:
tue_data.loc[tue_data[(np.isnan(tue_data["Headway (s)"])) & (tue_data["Hour"]==13)].index].head(5)

,Date,Lane,Lane Name,Direction,Direction Name,Speed (mph),Headway (s),Gap (s),Flags,Flag Text,Hour
94959,2018-02-06 13:35:37.000,2,NB_MID,1,North,29.205,NaN,NaN,2,Tuesday,13
95862,2018-02-06 13:50:39.030,2,NB_MID,1,North,43.495,NaN,NaN,2,Tuesday,13
96146,2018-02-06 13:55:17.050,2,NB_MID,1,North,38.525,NaN,NaN,2,Tuesday,13
219259,2018-02-13 13:00:17.030,2,NB_MID,1,North,44.739,NaN,NaN,2,Tuesday,13
219565,2018-02-13 13:05:26.000,2,NB_MID,1,North,32.310,NaN,NaN,2,Tuesday,13


In [51]:
data.loc[tue_data[(np.isnan(tue_data["Headway (s)"])) & (tue_data["Hour"]==13)].index].head(5)

,Date,Lane,Lane Name,Direction,Direction Name,Speed (mph),Headway (s),Gap (s),Flags,Flag Text,Hour
94959,2018-02-06 13:35:37.000,2,NB_MID,1,North,29.205,3.102,2.756,2,Tuesday,13
95862,2018-02-06 13:50:39.030,2,NB_MID,1,North,43.495,3.102,2.756,2,Tuesday,13
96146,2018-02-06 13:55:17.050,2,NB_MID,1,North,38.525,3.102,2.756,2,Tuesday,13
219259,2018-02-13 13:00:17.030,2,NB_MID,1,North,44.739,3.102,2.756,2,Tuesday,13
219565,2018-02-13 13:05:26.000,2,NB_MID,1,North,32.310,3.102,2.756,2,Tuesday,13


In [52]:
tue_data.loc[tue_data[(np.isnan(tue_data["Headway (s)"])) & (tue_data["Hour"]==17)].index].head(5)

,Date,Lane,Lane Name,Direction,Direction Name,Speed (mph),Headway (s),Gap (s),Flags,Flag Text,Hour
237558,2018-02-13 17:15:22.020,2,NB_MID,1,North,33.554,NaN,NaN,2,Tuesday,17
238104,2018-02-13 17:20:57.090,2,NB_MID,1,North,4.970,NaN,NaN,2,Tuesday,17
363568,2018-02-20 17:00:06.080,2,NB_MID,1,North,35.417,NaN,NaN,2,Tuesday,17
366348,2018-02-20 17:30:25.030,2,NB_MID,1,North,30.447,NaN,NaN,2,Tuesday,17
368759,2018-02-20 17:55:22.060,2,NB_MID,1,North,29.825,NaN,NaN,2,Tuesday,17


In [53]:
data.loc[tue_data[(np.isnan(tue_data["Headway (s)"])) & (tue_data["Hour"]==17)].index].head(5)

,Date,Lane,Lane Name,Direction,Direction Name,Speed (mph),Headway (s),Gap (s),Flags,Flag Text,Hour
237558,2018-02-13 17:15:22.020,2,NB_MID,1,North,33.554,2.9065,2.187,2,Tuesday,17
238104,2018-02-13 17:20:57.090,2,NB_MID,1,North,4.970,2.9065,2.187,2,Tuesday,17
363568,2018-02-20 17:00:06.080,2,NB_MID,1,North,35.417,2.9065,2.187,2,Tuesday,17
366348,2018-02-20 17:30:25.030,2,NB_MID,1,North,30.447,2.9065,2.187,2,Tuesday,17
368759,2018-02-20 17:55:22.060,2,NB_MID,1,North,29.825,2.9065,2.187,2,Tuesday,17


In [54]:
tue_data.loc[tue_data[(np.isnan(tue_data["Headway (s)"])) & (tue_data["Hour"]==18)].index].head(5)

,Date,Lane,Lane Name,Direction,Direction Name,Speed (mph),Headway (s),Gap (s),Flags,Flag Text,Hour
115619,2018-02-06 18:00:01.190,2,NB_MID,1,North,41.010,NaN,NaN,2,Tuesday,18
116459,2018-02-06 18:10:17.020,2,NB_MID,1,North,36.661,NaN,NaN,2,Tuesday,18
117279,2018-02-06 18:20:21.050,2,NB_MID,1,North,29.825,NaN,NaN,2,Tuesday,18
118095,2018-02-06 18:30:17.090,2,NB_MID,1,North,30.447,NaN,NaN,2,Tuesday,18
118822,2018-02-06 18:40:23.090,2,NB_MID,1,North,39.768,NaN,NaN,2,Tuesday,18


In [55]:
data.loc[tue_data[(np.isnan(tue_data["Headway (s)"])) & (tue_data["Hour"]==18)].index].head(5)

,Date,Lane,Lane Name,Direction,Direction Name,Speed (mph),Headway (s),Gap (s),Flags,Flag Text,Hour
115619,2018-02-06 18:00:01.190,2,NB_MID,1,North,41.010,2.79,2.228,2,Tuesday,18
116459,2018-02-06 18:10:17.020,2,NB_MID,1,North,36.661,2.79,2.228,2,Tuesday,18
117279,2018-02-06 18:20:21.050,2,NB_MID,1,North,29.825,2.79,2.228,2,Tuesday,18
118095,2018-02-06 18:30:17.090,2,NB_MID,1,North,30.447,2.79,2.228,2,Tuesday,18
118822,2018-02-06 18:40:23.090,2,NB_MID,1,North,39.768,2.79,2.228,2,Tuesday,18


# Week 3 Task 6

In [1]:
import pandas as pd
import numpy as np

In [2]:
data1083 = pd.read_csv("rawpvr_2018-02-01_28d_1083 TueFri.csv")
data1415 = pd.read_csv("rawpvr_2018-02-01_28d_1415 TueFri.csv")

In [3]:
data1083["Date"] = pd.to_datetime(data1083["Date"], format="%Y-%m-%d %H:%M:%S.%f")
data1415["Date"] = pd.to_datetime(data1415["Date"], format="%Y-%m-%d %H:%M:%S.%f")

data1083["Hour"] = data1083["Date"].dt.hour
data1415["Hour"] = data1415["Date"].dt.hour

fri_data1083 = data1083[(data1083["Direction Name"]=="North") & (data1083["Flags"]==5) & (data1083["Hour"]==17)]
fri_data1415 = data1415[(data1415["Direction Name"]=="NorthEast") & (data1415["Flags"]==5) & (data1415["Hour"]==17)]

In [4]:
avg_speed_fri_data1083 = fri_data1083.groupby("Lane Name").mean()["Speed (mph)"]
avg_speed_fri_data1415 = fri_data1415.groupby("Lane Name").mean()["Speed (mph)"]
print("Average speed in 1083: ", avg_speed_fri_data1083)
print("Average speed in 1415: ,", avg_speed_fri_data1415)

Average speed in 1083:  Lane Name
NB_MID    28.781308
NB_NS     24.889867
NB_OS     30.153217
Name: Speed (mph), dtype: float64
Average speed in 1415: , Lane Name
NE_NS    24.096575
NE_OS    27.159322
Name: Speed (mph), dtype: float64


In [9]:
speed_fri_list = list(avg_speed_fri_data1083) + list(avg_speed_fri_data1415)
avg_speed_fri_mph = sum(speed_fri_list)/len(speed_fri_list)
print("Average speed (mph): ", avg_speed_fri_mph)

avg_speed_fri_kph = avg_speed_fri_mph * 1.61
print("Average speed (kph): ", avg_speed_fri_kph)

time_hours = 4.86/avg_speed_fri_kph
time_minutes = time_hours*60
print("Time from site 1083 to site 1415 in hours", time_hours)
print("Time from site 1083 to site 1415 in minutes", time_minutes)

Average speed (mph):  27.016057646033993
Average speed (kph):  43.495852810114734
Time from site 1083 to site 1415 in hours 0.1117347904688015
Time from site 1083 to site 1415 in minutes 6.70408742812809


# Week 3 Task 7.1

In [1]:
import pandas as pd
import numpy as np

In [8]:
data = pd.read_csv("rawpvr_2018-02-01_28d_1083 TueFri.csv")
data["Date"] = pd.to_datetime(data["Date"])
data["Hour"] = data["Date"].dt.hour

tue_data = data[(data["Flags"]==2) & (data["Hour"]>=7) & (data["Hour"]<19)]
tue_data = tue_data.drop(["Hour"], axis=1)
tue_data.isna().sum()

Date                 0
Lane                 0
Lane Name            0
Direction            0
Direction Name       0
Speed (mph)          9
Headway (s)       2081
Gap (s)           3949
Flags                0
Flag Text            0
dtype: int64

## Approch 1 [sum(non-empty cells/all cells)]/len(all cells)

In [66]:
accuracy = 0
for i in tue_data.index:
    accuracy += (len(tue_data.loc[i]) - tue_data.loc[i].isna().sum())/len(tue_data.loc[i])

accuracy = accuracy/len(tue_data)*100
print("Row completeness accuracy: ", accuracy)

Row completeness accuracy:  99.6997389682972


## Approch 2 row incomplete / row complete

In [71]:
tue_data.isna().sum()

Date                 0
Lane                 0
Lane Name            0
Direction            0
Direction Name       0
Speed (mph)          9
Headway (s)       2081
Gap (s)           3949
Flags                0
Flag Text            0
dtype: int64

In [72]:
nan_data_len = len(tue_data[(np.isnan(tue_data["Speed (mph)"])) | (np.isnan(tue_data["Headway (s)"])) | (np.isnan(tue_data["Gap (s)"]))].index)
nan_data_len

3958

In [76]:
accuracy = ((len(tue_data)-nan_data_len)/len(tue_data))*100
print("Row completeness accuracy: ", accuracy)

Row completeness accuracy:  98.03206960845246
